In [1]:
import pandas as pd
import numpy as np
import random
import json
import pickle
import h5py
import numba

In [2]:
cm = pd.read_csv('final.csv')
cm['genres'] = cm.genres.str.split('/')
cm['tags'] = cm.tags.str.split('\|\|')
mymv = set(cm.movieId)

In [4]:
cm.iloc[:,0]

0        180065
1        180067
2        180071
3        180075
4        180077
          ...  
25108    209073
25109    209085
25110    209123
25111    209129
25112    209169
Name: movieId, Length: 25113, dtype: int64

In [ ]:
s = set()
for it in cm['rating']:
    s |= set([it])

In [3]:
with open('movie-id-map.pkl', 'rb') as f:
    mv2id = pickle.load(f)

In [23]:
mymv = list(mymv)
mv2id = {mymv[i]: i for i in range(len(mymv))}
with open('movie-id-map.pkl', 'wb') as of:
    pickle.dump(mv2id, of)

In [7]:
def calculate_similarity(cname: str):
    dist = set()
    for it in cm[cname]:
        dist |= set(it)
    print(f"len {len(dist)}")
    dist = list(dist)
    p2id = {dist[i]: i for i in range(len(dist))}
    psim = np.zeros((len(dist), len(mymv)), dtype='<f2')
    for mv, it in cm[['movieId', cname]].itertuples(index=False):
        for i in it:
            psim[p2id[i], mv2id[mv]] += 1
    print('done psim')
    return psim
    msim = np.zeros((len(mymv), len(mymv)), dtype='<f2')
    for i in range(len(mymv)):
        msim[:, i] += np.linalg.norm(psim[:, np.newaxis, i]-psim, axis=0)
        if i%1000 == 0:
            print(f'done{i}')
    return msim

In [ ]:
np.savez_compressed('director-similarity.npz', calculate_similarity('director'))

In [ ]:
def big_calculate_similarity(cname: str):
    dist = set()
    for it in cm[cname]:
        dist |= set(it)
    print(f"len {len(dist)}")
    dist = list(dist)
    p2id = {dist[i]: i for i in range(len(dist))}
    f = h5py.File('temp.hdf5', 'w')
    psim = f.create_dataset(cname+'-similarity', (len(dist), len(mymv)), dtype='f2')
    i = 0
    for mv, it in cm[['movieId', cname]].itertuples(index=False):
        psim[np.sort(np.array([p2id[i] for i in it])), mv2id[mv]] = 1
        i += 1
        if i%1000 == 0:
            print(f'psim {i}')
    print('done psim')
    f.close()
    return
    final = h5py.File('similarities.hdf5', 'w')
    msim = final.create_dataset(cname+'-similarity', (len(mymv), len(mymv)), dtype='f2')
    for i in range(len(mymv)):
        msim[:, i] += np.linalg.norm(psim[:, i][:, np.newaxis]-psim, axis=0)
        if i%1000 == 0:
            print(f'done{i}')
    f.close()
    final.close()

In [ ]:
np.savez_compressed('genre-similarity.npz', calculate_similarity('genres'))

In [9]:
def rating_similarity():
    rsim = np.zeros((len(mymv), len(mymv)), dtype='<f2')
    i = 0
    for mv, rt in cm[['movieId', 'rating']].itertuples(index=False):
        rsim[mv2id[mv], :] += rt
        rsim[:, mv2id[mv]] -= rt
        i += 1
        if i%1000 == 0:
            print('done', i)
    return rsim

In [ ]:
np.savez_compressed('rating-similarity.npz', rating_similarity())

In [ ]:
final['cast-similarity']

In [10]:
def half_similarity(cname: str):
    print('s')
    rf = h5py.File('/media/aayush/Acer/UbuntuStuff/temp.hdf5', 'r')
    print('l1')
    psim = rf[cname+'-similarity']
    print('l2')
    final = h5py.File('/media/aayush/Acer/UbuntuStuff/similarities.hdf5', 'w')
    print('l3')
    msim = final.create_dataset(cname+'-similarity', (len(mymv), len(mymv)), dtype='f2')
    print('l4')
    for i in range(len(mymv)-1):
        print('start', i)
        row = rf[cname+'-similarity'][:, i]
        for j in range(i+1, len(mymv)):
            print('mid', j)
            msim[i, j] = np.linalg.norm(row - rf[cname+'-similarity'][:, j])
    rf.close()
    final.close()

In [ ]:
big_calculate_similarity('cast')


In [ ]:
half_similarity('cast')

In [7]:
def other_calculate_similarity(cname: str):
    dist = set()
    for it in cm[cname].dropna():
        dist |= set(it)
    print(f"len {len(dist)}")
    dist = list(dist)
    p2id = {dist[i]: i for i in range(len(dist))}
    psim = np.zeros((len(dist), len(mymv)), dtype='f4')
    for mv, it in cm[['movieId', cname]].dropna().itertuples(index=False):
        psim[[p2id[i] for i in it], mv2id[mv]] += 1
    print('done psim')
    return psim
    msim = np.zeros((len(mymv), len(mymv)), dtype='<f2')
    for i in range(len(mymv)):
        msim[:, i] += np.linalg.norm(psim[:, np.newaxis, i]-psim, axis=0)
        print(f'done{i}')
    return msim

In [12]:
@numba.jit(nopython=True, parallel=True)
def other_half_similarity(sim, nmv, start, end):
    print('Enter')
    msim = np.zeros(shape=(nmv, nmv), dtype=numba.float32)
    for i in range(start, end):
        col = sim[:, i]
        for j in range(i+1, nmv):
            msim[i, j] += np.linalg.norm(col-sim[:, j])
        print('done', i)
    return msim

In [9]:
psim = other_calculate_similarity('tags')

len 24529
done psim


In [13]:
other_half_similarity(psim, len(mymv), 0, 10)

Enter
done 0
done 1
done 2
done 3
done 4
done 5
done 6
done 7
done 8
done 9


array([[0.       , 3.1622777, 2.828427 , ..., 2.6457512, 3.1622777,
        2.236068 ],
       [0.       , 0.       , 3.1622777, ..., 2.6457512, 3.1622777,
        2.236068 ],
       [0.       , 0.       , 0.       , ..., 2.6457512, 3.1622777,
        2.236068 ],
       ...,
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ]], dtype=float32)

In [19]:
def subset_similarity(cname: int, movies: list):
    subset = cm[cm.movieId.isin(set(movies))].dropna()
    dist = set()
    for it in subset[cname]:
        dist |= set(it)
    dist = list(dist)
    print('done set')
    p_to_ind = {dist[i]: i for i in range(len(dist))}
    movie_to_ind = {movies[i]: i for i in range(len(movies))}
    psim = np.zeros((len(dist), len(movies)), dtype='<f2')
    for mv, it in subset[['movieId', cname]].itertuples(index=False):
        psim[[p_to_ind[i] for i in it], movie_to_ind[mv]] += 1
    print('done psim')
    msim = np.zeros((len(movies), len(movies)), dtype='<f2')
    for i in range(len(movies)):
        msim[:, i] += np.linalg.norm(psim[:, np.newaxis, i]-psim, axis=0)
#         if i%100 == 0:
        print('msim', i)
    return msim

In [15]:
umv = random.sample(mymv, 3000)

In [ ]:
mat = subset_similarity('tags', umv)

In [11]:
list(range(10))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]